In [ ]:
!pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken

In [ ]:
!pip install dotenv

In [1]:
!pip install chainlit

In [3]:
!chainlit hello

In [ ]:
from langchain.llms import OpenAI

In [ ]:
from langchain import PromptTemplate
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/mykey.env')

False

In [2]:
# !pip install huggingface hub

In [ ]:
# Model Testing before creating chatbot in Chainlit

# template = """Medical Text: {abstract}

# Class: """
# prompt = PromptTemplate(
#     template=template,
#     input_variables=['question']
# )

# # user question
# question = "Write a description here."

# from langchain import HuggingFaceHub, LLMChain

# # initialize Hub LLM
# hub_llm = HuggingFaceHub(
#         repo_id='aipradeepd/Evolent_BERT_Classifier',
#     model_kwargs={'temperature':0}
# )

# # create prompt template > LLM chain
# llm_chain = LLMChain(
#     prompt=prompt,
#     llm=hub_llm
# )

# # ask the user question about the capital of France
# print(llm_chain.run(question))
# # initialize Hub LLM
# hub_llm = HuggingFaceHub(
#         repo_id='aipradeepd/Evolent_BERT_Classifier',
#     model_kwargs={'temperature':0}
# )

# # create prompt template > LLM chain
# llm_chain = LLMChain(
#     prompt=prompt,
#     llm=hub_llm
# )

# # ask the user question about the capital of France
# print(llm_chain.run(question))

In [ ]:
from chainlit import cl
from langchain import HuggingFaceHub,PromptTemplate,LLMChain

In [ ]:
model_id='aipradeepd/Evolent_BERT_Classifier'
evolent_model = HuggingFaceHub(repo_id='aipradeepd/Evolent_BERT_Classifier',model_kwargs={'temperature':0})


In [ ]:
template = """ You are a Medical Assistant that predicts the class of Medical test based on abstract received as a input


{abstract}
"""

In [ ]:
@cl_on_chat_start
def main():
  prompt = PromptTemplate(template=template,input_variables=['abstract'])
  model_chain = LLMChain(llm=evolent_model,prompt=prompt,verbose=True)

  cl.user_session.set('evolent_chain',model_chain)


In [ ]:
@cl.on_message
async def main(message:str):
  evolent_chain = cl.user_session.get('evolent_chain')
  res = await evolent_chain.acall(message,callbacks=[cl.AsyncLangchainCallbackHandler()])

  await cl.Message(content=res['text']).send()


==============================================================